# 01 Panel Fixed Effects + Clustered SE

Pooled vs two-way fixed effects and clustered standard errors.


## Table of Contents
- [Load panel and define variables](#load-panel-and-define-variables)
- [Pooled OLS baseline](#pooled-ols-baseline)
- [Two-way fixed effects](#two-way-fixed-effects)
- [Clustered standard errors](#clustered-standard-errors)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Causal notebooks focus on **identification**: what would have to be true for a coefficient to represent a causal effect.
You will practice:
- building a county-year panel,
- fixed effects (TWFE),
- clustered standard errors,
- DiD + event studies,
- IV/2SLS.


## Prerequisites (Quick Self-Check)
- Completed Part 02 (regression + robust SE).
- Basic familiarity with panels (same unit over time) and the idea of identification assumptions.

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Treating regression output as causal without stating identification assumptions.
- Using non-clustered SE when shocks are correlated within groups (e.g., states).

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/06_causal/01_panel_fixed_effects_clustered_se.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/06_causal/01_panel_fixed_effects_clustered_se.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Compare:
- pooled OLS (ignores panel structure)
- two-way fixed effects (county FE + year FE)
- robust vs clustered standard errors

This is still not causal by default. FE helps control time-invariant confounding, not everything.



## Primer: Panel + IV regression with `linearmodels` (FE, clustered SE, 2SLS)

This repo uses:
- `statsmodels` for classic OLS inference patterns, and
- `linearmodels` for **panel fixed effects** and **instrumental variables** (IV/2SLS).

The goal of this primer is to make you productive quickly (with the *minimum* theory needed to use the tools correctly). Deep math lives in the guides.

### Why `linearmodels`?

`linearmodels` provides clean APIs for:
- `PanelOLS`: fixed effects / TWFE
- `IV2SLS`: two-stage least squares

and it handles some panel-specific details (like absorbing FE) more naturally than `statsmodels`.

### Panel data shape (the #1 requirement)

Most panel estimators expect a **MultiIndex**:
- level 0: entity (e.g., county `fips`)
- level 1: time (e.g., `year`)

```python
# df has columns: fips, year, y, x1, x2, state, ...
df = df.copy()
df["fips"] = df["fips"].astype(str)
df["year"] = df["year"].astype(int)
df = df.set_index(["fips", "year"]).sort_index()
```

**Expected output / sanity check**
- `df.index.nlevels == 2`
- `df.index.is_monotonic_increasing` is `True`
- no duplicate index pairs: `df.index.duplicated().any()` is `False`

### TWFE model (PanelOLS)

Econometric form:

$$
Y_{it} = X_{it}'\\beta + \\alpha_i + \\gamma_t + \\varepsilon_{it}
$$

In code:

```python
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

y = df["y"].astype(float)
X = df[["x1", "x2"]].astype(float)
X = sm.add_constant(X, has_constant="add")

res = PanelOLS(y, X, entity_effects=True, time_effects=True).fit(cov_type="robust")
print(res.summary)
```

### Clustered SE (common in applied panel/DiD work)

If errors are correlated within clusters (e.g., state-level shocks), use clustered SE:

```python
clusters = df["state"]  # must align row-for-row with y/X index

res_cl = PanelOLS(y, X, entity_effects=True, time_effects=True).fit(
  cov_type="clustered",
  clusters=clusters,
)
```

**Expected output / sanity check**
- clustered SE are often larger than robust SE (not guaranteed, but common)
- always report the number of clusters: `clusters.nunique()`

### IV / 2SLS (IV2SLS)

Structural equation (endogeneity motivation):
$$
Y = \\beta X + W'\\delta + u, \\quad \\mathrm{Cov}(X,u)\\neq 0
$$

In code (one endogenous regressor):

```python
from linearmodels.iv import IV2SLS
import statsmodels.api as sm

y = df["y"].astype(float)
endog = df[["x_endog"]].astype(float)
exog = sm.add_constant(df[["x_exog1", "x_exog2"]].astype(float), has_constant="add")
instr = df[["z1", "z2"]].astype(float)

res_iv = IV2SLS(y, exog, endog, instr).fit(cov_type="robust")
print(res_iv.summary)
```

**Expected output / sanity check**
- `res_iv.params` contains coefficients for exog + endogenous variables
- `res_iv.first_stage` (if printed) shows instrument relevance diagnostics

### Common pitfalls (and quick fixes)

- **MultiIndex mismatch:** if `clusters` is not aligned to the same index as `y/X`, you’ll get errors or wrong results.
  - Fix: construct clusters from the same `df` after indexing/sorting.
- **Non-numeric dtypes:** strings in `X` silently break models.
  - Fix: `astype(float)` on model columns.
- **Missing data:** panels often have missing rows after merges/transforms.
  - Fix: build a modeling table with `.dropna()` for required columns.
- **Too few clusters:** cluster-robust inference is fragile with very small cluster counts.
  - Fix: treat p-values as fragile; report cluster count; consider alternative designs.


<a id="load-panel-and-define-variables"></a>
## Load panel and define variables

### Background
Panel regressions expect a clear unit index (county) and time index (year).
Before modeling, we build a **small, typed modeling table**:
- confirm `fips` and `year`,
- set a MultiIndex,
- create a few interpretable transforms (like logs).

Log transforms are common for heavy-tailed variables (income, rent) because they reduce scale and make multiplicative differences more linear.

### What you should see
- the DataFrame is indexed by `('fips','year')`.
- `log_income` and `log_rent` are finite (no -inf/inf).
- summary stats look plausible (rates roughly in [0,1]).

### Interpretation prompts
- Why might `log_income` be easier to interpret than raw income?
- What does a 0.01 change in `poverty_rate` represent?

### Goal
Load the county-year panel and build a small modeling table.



### Your Turn: Load panel (processed or sample)


In [ ]:
import numpy as np
import pandas as pd

path = PROCESSED_DIR / 'census_county_panel.csv'
if path.exists():
    df = pd.read_csv(path)
else:
    df = pd.read_csv(SAMPLE_DIR / 'census_county_panel_sample.csv')

# TODO: Ensure fips/year exist and build a MultiIndex
df['fips'] = df['fips'].astype(str)
df['year'] = df['year'].astype(int)
df = df.set_index(['fips', 'year'], drop=False).sort_index()

# Starter transforms
df['log_income'] = np.log(df['B19013_001E'].astype(float))
df['log_rent'] = np.log(df['B25064_001E'].astype(float))

df[['poverty_rate', 'unemployment_rate', 'log_income', 'log_rent']].describe()



<a id="pooled-ols-baseline"></a>
## Pooled OLS baseline

### Background
Pooled OLS treats each row as an independent observation and ignores that rows come from the same county over time.
It is a useful baseline, but it can be misleading when counties differ in unobserved, time-invariant ways (baseline poverty, institutions, geography).

### What you should see
- a `statsmodels` summary table.
- coefficients with HC3 robust SE.

### Interpretation prompts
- Interpret the sign and units of one coefficient in 2–4 sentences.
- List one plausible omitted variable that differs across counties and could confound this pooled relationship.

### Goal
Fit a pooled model that ignores FE.



### Your Turn: Fit pooled OLS


In [ ]:
import statsmodels.api as sm

y_col = 'poverty_rate'
x_cols = ['log_income', 'unemployment_rate']

tmp = df[[y_col] + x_cols].dropna().copy()
y = tmp[y_col].astype(float)
X = sm.add_constant(tmp[x_cols].astype(float), has_constant='add')

# TODO: Fit and print a summary (HC3 as a baseline)
res_pool = sm.OLS(y, X).fit(cov_type='HC3')
print(res_pool.summary())



<a id="two-way-fixed-effects"></a>
## Two-way fixed effects

### Background
Two-way fixed effects (TWFE) compares counties to themselves over time (county FE) while removing year-wide shocks (year FE).
This can reduce bias from time-invariant county differences.

### What you should see
- a `PanelOLS` summary.
- coefficients that may differ from pooled OLS (because identification uses within-county changes).

### Interpretation prompts
- Compare pooled vs TWFE: did the coefficient move? What story could explain the change?
- What variation identifies $\beta$ in TWFE (within county, across time)?

### Goal
Estimate a TWFE model:
- county FE (entity)
- year FE (time)



### Your Turn: Fit TWFE with PanelOLS


In [ ]:
from src.causal import fit_twfe_panel_ols

# TODO: Fit TWFE (robust SE)
res_twfe = fit_twfe_panel_ols(
    df,
    y_col=y_col,
    x_cols=x_cols,
    entity_effects=True,
    time_effects=True,
)
print(res_twfe.summary)



<a id="clustered-standard-errors"></a>
## Clustered standard errors

### Background
Even with TWFE, inference can be too optimistic if errors are correlated within groups.
A common choice here is clustering by state because counties in the same state share policies and shocks.

### What you should see
- a table comparing robust vs clustered standard errors.
- clustered SE are often larger (not guaranteed, but common).

### Interpretation prompts
- Which SE would you report for a state-level policy story and why?
- How many clusters do you have (unique states), and why does that matter?

### Goal
Re-fit TWFE with clustered SE.

Typical clustering choice here:
- by state (shared shocks/policies)



### Your Turn: Cluster by state and compare SE


In [ ]:
import pandas as pd
from src.causal import fit_twfe_panel_ols

# TODO: Compare robust vs clustered SE
res_cluster = fit_twfe_panel_ols(
    df,
    y_col=y_col,
    x_cols=x_cols,
    entity_effects=True,
    time_effects=True,
    cluster_col='state',
)

pd.DataFrame({'robust_se': res_twfe.std_errors, 'cluster_se': res_cluster.std_errors})



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# Expected output: (see notebook front matter)
# TODO: If you created a panel DataFrame, verify the indexing + core columns.
# Example (adjust variable names):
# assert isinstance(panel.index, pd.MultiIndex)
# assert panel.index.names[:2] == ['fips', 'year']
# assert panel['year'].astype(int).between(1900, 2100).all()
# assert panel['fips'].astype(str).str.len().eq(5).all()
#
# TODO: Write 2-3 sentences:
# - What is the identification assumption for your causal estimate?
# - What diagnostic/falsification did you run?
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Load panel and define variables</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_panel_fixed_effects_clustered_se — Load panel and define variables
import numpy as np
import pandas as pd

path = PROCESSED_DIR / 'census_county_panel.csv'
if path.exists():
    df = pd.read_csv(path)
else:
    df = pd.read_csv(SAMPLE_DIR / 'census_county_panel_sample.csv')

df['fips'] = df['fips'].astype(str)
df['year'] = df['year'].astype(int)
df = df.set_index(['fips', 'year'], drop=False).sort_index()

df['log_income'] = np.log(df['B19013_001E'].astype(float))
df['log_rent'] = np.log(df['B25064_001E'].astype(float))
df[['poverty_rate', 'log_income', 'unemployment_rate']].describe()
```

</details>

<details><summary>Solution: Pooled OLS baseline</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_panel_fixed_effects_clustered_se — Pooled OLS baseline
import statsmodels.api as sm

tmp = df[['poverty_rate', 'log_income', 'unemployment_rate']].dropna().copy()
y = tmp['poverty_rate'].astype(float)
X = sm.add_constant(tmp[['log_income', 'unemployment_rate']], has_constant='add')
res = sm.OLS(y, X).fit(cov_type='HC3')
print(res.summary())
```

</details>

<details><summary>Solution: Two-way fixed effects</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_panel_fixed_effects_clustered_se — Two-way fixed effects
from src.causal import fit_twfe_panel_ols

res_twfe = fit_twfe_panel_ols(
    df,
    y_col='poverty_rate',
    x_cols=['log_income', 'unemployment_rate'],
    entity_effects=True,
    time_effects=True,
)
print(res_twfe.summary)
```

</details>

<details><summary>Solution: Clustered standard errors</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_panel_fixed_effects_clustered_se — Clustered standard errors
from src.causal import fit_twfe_panel_ols

res_cluster = fit_twfe_panel_ols(
    df,
    y_col='poverty_rate',
    x_cols=['log_income', 'unemployment_rate'],
    entity_effects=True,
    time_effects=True,
    cluster_col='state',
)

pd.DataFrame({'robust_se': res_twfe.std_errors, 'cluster_se': res_cluster.std_errors})
```

</details>

